# Microsoft Movie Studio Analysis

**Authors:** Joseph Ngige
***

## Overview
***
Microsoft's new movie studio aims to identify the most successful movie genres at the box office. By analyzing data from [IMDb website](https://www.imdb.com/) and [Box Office Mojo](https://www.boxofficemojo.com/) websites, I determined that Action, Adventure, and Sci-Fi combinations consistently achieve the highest domestic and foreign gross revenues, as well as the most votes. Adventure, in particular, is a common element in top-performing genres. Therefore, I would recommend focusing on Action, Adventure, and Sci-Fi movies, and also suggest considering Adventure paired with Animation or Fantasy. Additionally, the combination of Adventure, Animation, and Comedy shows strong potential for financial success.
***

## Business Problem
***
Microsoft sees all the big companies creating original video content and wants to get in on the fun by creating a new movie studio. However, they lack experience in making movies. They would therefore want to explore which types of movies are currently the most successful at the box office and translating those findings into actionable insights for their new movie studio. To determine the most profitable movie genres, both domestic and foreign sales data for top movie genres was analyzed, as well as the average ratings and number of votes for each genre, to compare their popularity with financial success.
***

## Data Understanding
***
The data analyzed came from an SQLite database and a CSV file sourced from IMDb and Box Office Mojo websites, respectively. IMDb is a widely-used database for movies, TV shows, video games, and streaming content, while Box Office Mojo tracks global box office revenue, including domestic and international grosses. Using the movie_basics and movie_ratings tables from the IMDb database and the CSV data from Box Office Mojo, I focused on determining the most successful genres by analyzing domestic and foreign gross sales, average ratings, and number of votes.

In [1]:
# Import standard packages
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Load the IMDB Database

First, load the IMDB SQLite database and inspect the relevant tables (movie_basics and movie_ratings).

In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('data/im.db')
# Initialize a cursor
cursor = conn.cursor()

In [3]:
# List the tables in the SQLite database
tables_query = "SELECT name AS 'Table Names' FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(tables_query, conn)
tables

,Table Names
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [4]:
# Load the relevant tables, movie_basics and movie_ratings tables
df_movie_basics = pd.read_sql_query("SELECT * FROM movie_basics;", conn)
df_movie_ratings = pd.read_sql_query("SELECT * FROM movie_ratings;", conn)

In [5]:
# Display the first few rows of movie_basics table
df_movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [6]:
# Check the movie_basics dataframe information
df_movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [7]:
# Display the first few rows of movie_ratings table
df_movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [8]:
# Check the movie_ratings dataframe information
df_movie_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [9]:
# Close the SQLite database connection
conn.close()

### Load the Box Office Mojo Data

Next, load the Box Office Mojo CSV file and inspect its structure.

In [10]:
# Load the Box Office Mojo CSV file and display the first few rows
df_bom_movie = pd.read_csv('data/bom.movie_gross.csv')
df_bom_movie.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [11]:
# Check the df_bom_movie dataset information
df_bom_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


### Initial Data Insights

IMDB Tables (movie_basics and movie_ratings):

* These tables provide information on movie titles, genres, release years, runtime, and ratings.
* The movie_id can be used to join these tables for combined insights.
* The movie_basics table has some missing values.

Box Office Mojo Data:

* This table provides box office performance data, including domestic and international earnings.
* The box office data has some missing values.
***

## Data Preparation
***
After obtaining initial data insights, I cleaned the data to ensure integrity by addressing missing values. Critical columns like movie_basics and movie_ratings were cleaned by removing rows with missing data, and financial figures were imputed with zeros where appropriate. Data types were standardized for accurate analysis, and financial figures were normalized to millions for readability. I then merged the datasets using common attributes, such as movie titles and IDs, to create a unified dataset containing all essential information. Non-essential columns, such as "studio" and "original_title," were dropped to streamline the dataset.

### Data Cleaning

#### Movie Basics Data

In [12]:
# Check for missing values in movie_basics
missing_values_basics = df_movie_basics.isnull().sum()
missing_values_basics

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

There are missing values in the movie_basics table:

* **original_title**: Some missing row values (21).
* **runtime_minutes**: A significant number of missing row values (31,739).
* **genres**: Some missing row values (5,408).

In [13]:
# Cleaning Movie Basics data
# Drop the original_title column as it is not needed for the analysis
movie_basics_clean = df_movie_basics.drop(columns=['original_title'])

# Fill missing runtime_minutes with 0
movie_basics_clean['runtime_minutes'] = movie_basics_clean['runtime_minutes'].fillna(0)

# Drop rows with missing genres as they are critical for the analysis
movie_basics_clean = movie_basics_clean.dropna(subset=['genres'])

# Convert data types
movie_basics_clean['start_year'] = movie_basics_clean['start_year'].astype(int)
movie_basics_clean['runtime_minutes'] = movie_basics_clean['runtime_minutes'].astype(int)

movie_basics_clean

,movie_id,primary_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,2013,175,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,114,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,122,Drama
3,tt0069204,Sabse Bada Sukh,2018,0,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,2017,80,"Comedy,Drama,Fantasy"
...,...,...,...,...,...
146138,tt9916428,The Secret of China,2019,0,"Adventure,History,War"
146139,tt9916538,Kuambil Lagi Hatiku,2019,123,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,0,Documentary
146141,tt9916706,Dankyavar Danka,2013,0,Comedy


In [14]:
# Checking Movie_Basics_Clean information
movie_basics_clean.info()

# Shows the cleaned data has no missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140736 entries, 0 to 146143
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   movie_id         140736 non-null  object
 1   primary_title    140736 non-null  object
 2   start_year       140736 non-null  int32 
 3   runtime_minutes  140736 non-null  int32 
 4   genres           140736 non-null  object
dtypes: int32(2), object(3)
memory usage: 5.4+ MB


##### Comments on Movie Basics Data Cleaning

* **original_title**: Dropped the column since it is not critical for the analysis focused on understanding successful film types at the box office.
* **runtime_minutes**: Filled missing values with 0. This maintains numerical consistency and allows us to keep the records. While not entirely realistic, this approach ensures that the analysis is not disrupted by missing runtime data.
* **genres**: Dropped rows with missing genres as they are critical for the analysis. Missing genre information would hinder the ability to categorize and analyze movies effectively.